# Forecasting Pension Cash Flow with NumPy

 Teake Nutma, May 2025

Hi PyGrunn 2025! 👋

## Introduction

This notebook computes the cash flow forecast for a temporary retirement pension with:

- a capital of €100.000,-
- payments that start in two years, run for eight, and stop in ten
- payments that occur at the end of the year
- a yearly interest of 5%
- and mortality rate that starts of at 2% and increases linearly to 20%.

We'll reproduce the following table with NumPy:

| **Year**  | **Unit payment** | **Chance of survival** | **Cumulative chance of survival** | **Expected unit payment** | **Interest** | **Cumulative interest** | **Discounted expected unit payment** | **Forecast** |
| --------- | ---------------- | ---------------------- | --------------------------------- | ------------------------- | ------------ | ----------------------- | ------------------------------------ | ------------ |
| **1**     | 0                | 98%                    | 98%                               | 0,00                      | 5%           | 5%                      | 0,00                                 | € 0          |
| **2**     | 0                | 96%                    | 94%                               | 0,00                      | 5%           | 10%                     | 0,00                                 | € 0          |
| **3**     | 1                | 94%                    | 88%                               | 0,88                      | 5%           | 16%                     | 0,76                                 | € 24.360     |
| **4**     | 1                | 92%                    | 81%                               | 0,81                      | 5%           | 22%                     | 0,67                                 | € 22.412     |
| **5**     | 1                | 90%                    | 73%                               | 0,73                      | 5%           | 28%                     | 0,57                                 | € 20.170     |
| **6**     | 1                | 88%                    | 64%                               | 0,64                      | 5%           | 34%                     | 0,48                                 | € 17.750     |
| **7**     | 1                | 86%                    | 55%                               | 0,55                      | 5%           | 41%                     | 0,39                                 | € 15.265     |
| **8**     | 1                | 84%                    | 47%                               | 0,47                      | 5%           | 48%                     | 0,32                                 | € 12.823     |
| **9**     | 1                | 82%                    | 38%                               | 0,38                      | 5%           | 55%                     | 0,25                                 | € 10.515     |
| **10**    | 1                | 80%                    | 31%                               | 0,31                      | 5%           | 63%                     | 0,19                                 | € 8.412      |
| **Total** |                  |                        |                                   |                           |              |                         | **3,63**                             | € 131.706    |


The actuarial factor $F$ is equal to the total discounted expected unit payment. The simplified formula we'll use for computing $F$ is as follows:
$$
F_i = \sum_{t=0}^\infty \sum_{j=1}^n Q^{\textrm{cum}}_{ij}(t) P_j(t) D(t)
$$
where $Q^{\textrm{cum}}$ are $2 \times 2$ cumulative transition matrices, $P$ are the unit payment vectors, and $D$ is the discounted value based on an interest curve. The indices $i,j$ run over the possible $n=2$ states, namely participant alive ($i=1$) and participant deceased ($i=2$).

Ok, that's enough talk for now. Let's start by importing NumPy and the `itertools` package from the stdlib. We'll also restrict to only two decimals for printouts.

In [1]:
import numpy as np
import itertools as it

np.set_printoptions(precision=2)

This notebook is structured as follows:

- We'll start by computing the discount value $D$,
- move on to the unit payment vectors $P$,
- then the cumulative transition matrices $Q^{\textrm{cum}}$,
- and finish by putting that all together for the factor $F$.

## Interest

Let's start by putting in the fixed 5% interest by hand:

In [2]:
i = np.linspace(0.05, 0.05, num=10)
i

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

We need to compute the year-by-year accumulated interest. We can do this very naively with a loop:

In [3]:
i_cum = []
prev = 1.0
for x in i + 1.0:
    i_cum.append(prev := x * prev)
np.array(i_cum) - 1.0

array([0.05, 0.1 , 0.16, 0.22, 0.28, 0.34, 0.41, 0.48, 0.55, 0.63])

While this makes use of [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) with the addition and subtraction of `1.0`, we can do clearly better and get rid of the explicit loop with [itertools.accumulate](https://docs.python.org/3/library/itertools.html#itertools.accumulate):

In [4]:
np.array(list(it.accumulate(i + 1.0, np.multiply))) - 1.0

array([0.05, 0.1 , 0.16, 0.22, 0.28, 0.34, 0.41, 0.48, 0.55, 0.63])

But [itertools.accumulate](https://docs.python.org/3/library/itertools.html#itertools.accumulate) still evaluates the loop in the Python interpreter. We can do one better still, and use a NumPy [universal function](https://numpy.org/doc/stable/reference/ufuncs.html) to push the computation down to the C level:

In [5]:
i_cum = np.multiply.accumulate(i + 1.0) - 1.0
i_cum

array([0.05, 0.1 , 0.16, 0.22, 0.28, 0.34, 0.41, 0.48, 0.55, 0.63])

### Speed comparison

Not only is `np.multiply.accumulate` more concise, it is also faster:

In [6]:
%timeit np.array(list(it.accumulate(i + 1.0, np.multiply))) - 1.0

13.7 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
%timeit np.multiply.accumulate(i + 1.0) - 1.0

2.03 µs ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


That's about 5 times faster! Because the input array is small, there's relatively much time spent on overhead. With larger arrays, the speed difference becomes more prominent.

### Discounted value

And now finally the discounted value per year becomes the following:

In [8]:
D = 1.0 / (i_cum + 1.0)
D

array([0.95, 0.91, 0.86, 0.82, 0.78, 0.75, 0.71, 0.68, 0.64, 0.61])

## Unit payments

Okay, so let's now construct the list of unit payment vectors by hand. (You'd normally construct this list from a given start date and a given end date).

The first two years are non-paying, while the last eight years are paying. But only in state 1 (alive); state 2 (deceased) does not give a payment. A single payment vector $p_i$ is thus `[1, 0]`. To build the full list $P_i(t)$ of payment vectors for all ten years, we can simply take the outer product of the payment vector with a list $\delta_t$ containing zeroes for non-paying years, and ones for paying years:

$$
P_i(t) = \delta_t \otimes p_i
$$

This can be done with [numpy.outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html):

In [9]:
p = [1,0]
d = [0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
P = np.outer(d, p)
P

array([[0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

But there's more than one way to skin a cat. We can also use the [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) method to compute the outer product:

In [11]:
np.einsum("t,i->ti", d, p)

array([[0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

Here the first argument `"t,i->ti"` indicates the indices of the input arguments (`"t,i"`) before the arrow and the indices of the output matrix (`"ti"`) after the arrow. Note that for simple cases like outer products between two vectors, there's no need to use [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) instead of [numpy.outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html). But for more complicated things [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) can be a life saver, as we'll see later on.

## Transition probability matrices

Okay, so we'll now move on to computing the transition probability matrices $Q$ from the given mortality rates $q$. You'd normally read the mortality rates from a so-called _mortality table_, but here we'll put them in by hand:

In [12]:
q = np.linspace(0.02, 0.20, num=10)
q

array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ])

The full transition probably matrix $Q_{ij}$ is given by:

$$
Q_{ij} = \begin{pmatrix} 1 - q & q \\ 0 & 1 \end{pmatrix}
$$

That is:

- $Q_{11}$ is the chance per year to keep on living, namely $1 - q$,
- $Q_{12}$ is the chance per year to die, $q$,
- $Q_{21}$ is the chance per year to be resurrected,
- $Q_{22}$ is the chance per year to stay dead if you're already dead.

Note that the rows add up to $1$; this is a so-called [stochastic matrix](https://en.wikipedia.org/wiki/Stochastic_matrix).

So now for each all mortality rates we have to construct the corresponding transition matrix. We can't use the outer product trick we used for the payments, because the mortality rates change from year to year. So we do this by constructing four lists for each of the entries of the matrix, and then re-arrange them:

In [13]:
Q_11 = 1.0 - q
Q_12 = q
Q_21 = np.zeros_like(q)
Q_22 = np.ones_like(q)
Q = np.column_stack([Q_11, Q_12, Q_21, Q_22])
Q

array([[0.98, 0.02, 0.  , 1.  ],
       [0.96, 0.04, 0.  , 1.  ],
       [0.94, 0.06, 0.  , 1.  ],
       [0.92, 0.08, 0.  , 1.  ],
       [0.9 , 0.1 , 0.  , 1.  ],
       [0.88, 0.12, 0.  , 1.  ],
       [0.86, 0.14, 0.  , 1.  ],
       [0.84, 0.16, 0.  , 1.  ],
       [0.82, 0.18, 0.  , 1.  ],
       [0.8 , 0.2 , 0.  , 1.  ]])

This is still not quite right; the ordering is ok but the entires are not $2 \times 2$ matrices yet. We can change the shape without copying the data with [numpy.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html):

In [14]:
Q = Q.reshape(-1, 2, 2)
Q

array([[[0.98, 0.02],
        [0.  , 1.  ]],

       [[0.96, 0.04],
        [0.  , 1.  ]],

       [[0.94, 0.06],
        [0.  , 1.  ]],

       [[0.92, 0.08],
        [0.  , 1.  ]],

       [[0.9 , 0.1 ],
        [0.  , 1.  ]],

       [[0.88, 0.12],
        [0.  , 1.  ]],

       [[0.86, 0.14],
        [0.  , 1.  ]],

       [[0.84, 0.16],
        [0.  , 1.  ]],

       [[0.82, 0.18],
        [0.  , 1.  ]],

       [[0.8 , 0.2 ],
        [0.  , 1.  ]]])

The `-1` means _"automatically compute the size of this axis such that number of entries match"_.

### Cumulative transition matrices

Now that we have the transition probability matrices, we can proceed to compute the cumulative transition probability matrices. The former describe the chances of making it from year $t$ to year $t=1$, while the latter describe the chances of making it from year $1$ to year $t+1$.

We computed the cumulative interest with the [accumulate](https://numpy.org/doc/stable/reference/generated/numpy.ufunc.accumulate.html) method of the universal [multiply](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html) NumPy function. The difference here is that the entries are matrices instead of scalars, so we'll use the [matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) universal function:

In [15]:
np.matmul.accumulate(Q)

<class 'RuntimeError'>: Reduction not defined on ufunc with signature

Well, that didn't work. Turns out you can't use accumulate with functions that operator on non-scalars!

So let's revert to the [itertools accumulate](https://docs.python.org/3/library/itertools.html#itertools.accumulate) version:

In [16]:
Q_cum = np.array(list(it.accumulate(Q, np.matmul)))
Q_cum

array([[[0.98, 0.02],
        [0.  , 1.  ]],

       [[0.94, 0.06],
        [0.  , 1.  ]],

       [[0.88, 0.12],
        [0.  , 1.  ]],

       [[0.81, 0.19],
        [0.  , 1.  ]],

       [[0.73, 0.27],
        [0.  , 1.  ]],

       [[0.64, 0.36],
        [0.  , 1.  ]],

       [[0.55, 0.45],
        [0.  , 1.  ]],

       [[0.47, 0.53],
        [0.  , 1.  ]],

       [[0.38, 0.62],
        [0.  , 1.  ]],

       [[0.31, 0.69],
        [0.  , 1.  ]]])

This is the most elegant way to do cumulative matrix multiplication with NumPy, but as noted before, it is not super-fast because the loop takes place in the Python interpreter. If speed is a must, it might be worthwhile to look at JIT tooling like [Numba](https://numba.pydata.org/) or [Jax](https://docs.jax.dev/) (but that's out of scope for this notebook).

## Actuarial factor

Okay, so now we have all ingredients for computing the acturial factor and cash flow, namely $Q^{\textrm{cum}}$, $P$, and $D$. 
Recall that the factor is given by:
$$
F_i = \sum_{t=0}^\infty \sum_{j=1}^n Q^{\textrm{cum}}_{ij}(t) P_j(t) D(t)
$$
Let's try to put this together and compute the summand $S_i(t) = \sum_{j=1}^n Q^{\textrm{cum}}_{ij}(t) P_j(t) D(t)$. This is the matrix product of $Q^{\textrm{cum}}$ and $P$, multiplied by $D$ for each time $t$:

In [17]:
S = Q_cum @ P * D

<class 'ValueError'>: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 2)

This didn't work because NumPy doesn't know it do should this element-wise for each year $t$. 
So tell it explicitly to do so:

In [18]:
S = np.array([Q_cum[t] @ P[t] * D[t] for t in range(10)])
S

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.76, 0.  ],
       [0.67, 0.  ],
       [0.57, 0.  ],
       [0.48, 0.  ],
       [0.39, 0.  ],
       [0.32, 0.  ],
       [0.25, 0.  ],
       [0.19, 0.  ]])

But this contains a loop, and loops are evil! So let's use [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) to compute the summand. The input indices are as follows:

- $Q^{\textrm{cum}}$: `tij` (the first index corresponds to the time, the second and third to the state),
- $P$: `tj` (first index is again time, the second is used in the matrix product with $Q^{\textrm{cum}}$,
- $D$: `t`,

while the output indices of $S$ are `ti`. So the Einstein summation can be done like so:

In [19]:
S = np.einsum("tij,tj,t->ti", Q_cum, P, D)
S

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.76, 0.  ],
       [0.67, 0.  ],
       [0.57, 0.  ],
       [0.48, 0.  ],
       [0.39, 0.  ],
       [0.32, 0.  ],
       [0.25, 0.  ],
       [0.19, 0.  ]])

### Speed comparison

The Einstein summation might be a little bit trickier to wrap your head around, but it's about 10 times faster even for short arrays:

In [20]:
%timeit np.array([Q_cum[t] @ P[t] * D[t] for t in range(10)])

26.5 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [21]:
%timeit np.einsum("tij,tj,t->ti", Q_cum, P, D)

3.62 µs ± 53.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Actuarial factor

We're now finally in a position to compute the actuarial factor:

In [22]:
F = S.sum(axis=0)
F

array([3.63, 0.  ])

The first entry corresponds to the alive state, i.e. if the participant is alive at $t=0$ the factor is $3.12$. If the second entry corresponds to the deceased state, thus if the participant is deceased at $t=0$ the factor is $0$. Since you know the participant is alive at $t=0$, it is sufficient to only look at the factor of the actual state.

## Forecast

Recall that the capital is €100.000,-. The yearly benefit $B$ is the capital divided by the actuarial factor $F$:

In [23]:
B = 100_000 / F[0]
B

np.float64(27546.068570616226)

Expected payments in the cash flow are undiscounted, so let's remove the discount:

In [24]:
forecast = B * S[:, 0] / D
forecast

array([    0.  ,     0.  , 24360.42, 22411.59, 20170.43, 17749.98,
       15264.98, 12822.58, 10514.52,  8411.61])

We've used a bit of [fancy indexing](https://numpy.org/doc/stable/user/basics.indexing.html) here. The colon here means _"select all elements of this axis"_.

The forecast is the same as was listed in the introduction, so yay!
The expected total payment over all years is then:

In [25]:
forecast.sum()

np.float64(131706.1102579442)